In [1]:
import pandas as pd
import sqlite3
import datetime
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import PercentFormatter
import math
from urllib.parse import urlparse, parse_qs
mpl.rcParams['figure.dpi'] = 200
okabe_ito = ["#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
sns.set_context('paper')
sns.set_palette(sns.color_palette(okabe_ito))
#https://stackoverflow.com/a/45846841
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])
def make_cdf(series):
    return series.value_counts(normalize=True).sort_index().cumsum()
def make_pdf(series):
    return series.value_counts(normalize=True).sort_index()

In [3]:
df = pd.read_csv('quic_connect_times_and_plt.csv', sep=",", header=None, names=['msm_id','website','access_tech','1rtt_plt','1rtt_connect','1rtt_sec_connect','0rtt_plt','0rtt_connect','0rtt_sec_connect'])

In [5]:
df['1rtt_conn_delta'] = df['1rtt_connect'] - df['1rtt_sec_connect']
df['0rtt_conn_delta'] = df['0rtt_connect'] - df['0rtt_sec_connect']
df

,msm_id,website,access_tech,1rtt_plt,1rtt_connect,1rtt_sec_connect,0rtt_plt,0rtt_connect,0rtt_sec_connect,1rtt_conn_delta,0rtt_conn_delta
0,3ea66ee4-9ad2-4dce-9388-33c5a62b6023,www.example.org,dsl,193.1,66.2,65.7,192.9,59.8,59.3,0.5,0.5
1,c5ebaafb-55e5-4b61-bac2-171da4776390,www.example.org,cable,147.4,37.1,36.6,129.3,29.9,29.3,0.5,0.6
2,09cfbcad-6f56-4834-b551-41d9755f27e4,www.example.org,fiber,106.2,23.4,22.9,99.9,18.8,18.3,0.5,0.5
3,485369bb-9959-4db0-8aad-04f3dc272228,www.example.org,4g,279.5,103.2,102.6,271.4,96.4,95.9,0.6,0.5
4,b2c5b01b-6f33-47d3-b93f-6013444f7fed,www.example.org,4g-medium,304.9,118.4,117.9,296.3,111.3,110.8,0.5,0.5
5,15a30966-be43-4ef5-9899-e5a2c3bcddcb,www.wikipedia.org,dsl,726.0,66.5,66.0,689.1,59.9,59.4,0.5,0.5
6,5fd5e326-e5fd-42f9-8060-07a1c38264f8,www.wikipedia.org,cable,631.2,36.7,36.2,641.1,29.6,29.1,0.5,0.5
7,3da4628f-2880-4bdf-86c1-161adbc44a46,www.wikipedia.org,fiber,639.0,23.5,23.1,600.7,18.8,18.3,0.4,0.5
8,5855bc07-0706-4270-a2fc-febb105f6b57,www.wikipedia.org,4g,761.2,102.9,102.4,790.3,96.2,95.6,0.5,0.6
9,c44186df-7be0-4af2-9e79-d35f0c4e11a5,www.wikipedia.org,4g-medium,800.3,117.4,116.9,677.2,111.4,110.9,0.5,0.5


In [9]:
df['conn_diff_to_early_data'] = df['1rtt_connect'] - df['0rtt_connect']
df['sec_conn_diff_to_early_data'] = df['1rtt_sec_connect'] - df['0rtt_sec_connect']
df

,msm_id,website,access_tech,1rtt_plt,1rtt_connect,1rtt_sec_connect,0rtt_plt,0rtt_connect,0rtt_sec_connect,1rtt_conn_delta,0rtt_conn_delta,conn_diff_to_early_data,sec_conn_diff_to_early_data
0,3ea66ee4-9ad2-4dce-9388-33c5a62b6023,www.example.org,dsl,193.1,66.2,65.7,192.9,59.8,59.3,0.5,0.5,6.4,6.4
1,c5ebaafb-55e5-4b61-bac2-171da4776390,www.example.org,cable,147.4,37.1,36.6,129.3,29.9,29.3,0.5,0.6,7.2,7.3
2,09cfbcad-6f56-4834-b551-41d9755f27e4,www.example.org,fiber,106.2,23.4,22.9,99.9,18.8,18.3,0.5,0.5,4.6,4.6
3,485369bb-9959-4db0-8aad-04f3dc272228,www.example.org,4g,279.5,103.2,102.6,271.4,96.4,95.9,0.6,0.5,6.8,6.7
4,b2c5b01b-6f33-47d3-b93f-6013444f7fed,www.example.org,4g-medium,304.9,118.4,117.9,296.3,111.3,110.8,0.5,0.5,7.1,7.1
5,15a30966-be43-4ef5-9899-e5a2c3bcddcb,www.wikipedia.org,dsl,726.0,66.5,66.0,689.1,59.9,59.4,0.5,0.5,6.6,6.6
6,5fd5e326-e5fd-42f9-8060-07a1c38264f8,www.wikipedia.org,cable,631.2,36.7,36.2,641.1,29.6,29.1,0.5,0.5,7.1,7.1
7,3da4628f-2880-4bdf-86c1-161adbc44a46,www.wikipedia.org,fiber,639.0,23.5,23.1,600.7,18.8,18.3,0.4,0.5,4.7,4.8
8,5855bc07-0706-4270-a2fc-febb105f6b57,www.wikipedia.org,4g,761.2,102.9,102.4,790.3,96.2,95.6,0.5,0.6,6.7,6.8
9,c44186df-7be0-4af2-9e79-d35f0c4e11a5,www.wikipedia.org,4g-medium,800.3,117.4,116.9,677.2,111.4,110.9,0.5,0.5,6.0,6.0


In [10]:
df[df['1rtt_plt'] < df['0rtt_plt']]

,msm_id,website,access_tech,1rtt_plt,1rtt_connect,1rtt_sec_connect,0rtt_plt,0rtt_connect,0rtt_sec_connect,1rtt_conn_delta,0rtt_conn_delta,conn_diff_to_early_data,sec_conn_diff_to_early_data
6,5fd5e326-e5fd-42f9-8060-07a1c38264f8,www.wikipedia.org,cable,631.2,36.7,36.2,641.1,29.6,29.1,0.5,0.5,7.1,7.1
8,5855bc07-0706-4270-a2fc-febb105f6b57,www.wikipedia.org,4g,761.2,102.9,102.4,790.3,96.2,95.6,0.5,0.6,6.7,6.8
10,0d066e24-d4a7-4a48-a351-c43612f1ca47,www.instagram.com,dsl,1739.6,67.4,66.9,1741.0,59.4,58.9,0.5,0.5,8.0,8.0
